# **Optical camera communications image processing for receiver**

> Python script for the image processing part of the receiver of an optical camera comuunication system.

> The objective is to demodulate a message, using on-off keying modulation/demodulation.

> The python script will import an image and try to demodulate it by image processing methods.

> The python script will be executed on a Raspberry pi 4 or a pc.

# **Necessary imports**

In [2]:
import numpy as np
import cv2
import sys
import cv2
import matplotlib.pyplot as plt

# **The code**

In [13]:
# Logical variables for control
start = 0 # start value must be 0
end = 0 # start value must be 0

# Lists for the transmited message
start_list = []
start_sequence = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
end_sequence = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
message_list = []
N = len(start_sequence)

# Start the video feed
cap = cv2.VideoCapture(1)

if (cap.isOpened() == False):
    print("Error opening video camera")

# Read until video is completed
while(cap.isOpened()):
     
# Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        
        # If we received the message
        if end == 1:
            print("Message has been received! Getting message ready to be viewed.")
            break
        
        # Grayscale image and thresholding
        grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        (thresh, bnwFrame) = cv2.threshold(grayframe, 127, 255, cv2.THRESH_BINARY)

        # # Just for testing
        # counting the number of pixels
        # number_of_white_pix = np.sum(bnwFrame == 255)
        # number_of_black_pix = np.sum(bnwFrame == 0)
        # print('Number of white pixels:', number_of_white_pix)
        # print('Number of black pixels:', number_of_black_pix)
        # print(frame.shape)

        # detection procedure and/or frame cropping

        # Cropping: extract useful part of the frame (manually at first)
        y_min = 60
        y_max = 360
        x_min = 360
        x_max = 665
        useful_frame = bnwFrame[y_min:y_max, x_min:x_max]
        # # for testing
        # cv2.imshow('frame', useful_frame)
        # print(bnwFrame.shape)
        # print(useful_frame.shape)

        # # find the new image resolution
        row_num = y_max - y_min # height
        col_num = x_max - x_min # length


        if start == 0:
            # put the data of the frame inside the start_list
            # for each row with step == thickness
            thickness = 40 # pixels (change manually)
            threshold = 40 # pixels (change manually)
            for y in range(int(thickness/2), int(row_num - thickness/2), thickness):
                row = useful_frame[y, 0:col_num-1] # get the row
                holder = np.sum(row == 255) # add to holder the sum of white pixels
                if holder > threshold:
                    start_list.append(1)
                else:
                    start_list.append(0)
            
            # if list_size >= 30 check the last 30 elements
            # if last 30 elements are the starting sequence start = 1
            if len(start_list) >= N:
                if start_list[-N:] == start_sequence:
                    start = 1
        elif start == 1:
            # start storing the bits to the message list

            # for each row with step == thickness
            thickness = 10 # pixels (change manually)
            threshold = 100 # pixels (change manually)
            for y in range(int(thickness/2), int(row_num - thickness/2), thickness):
                row = useful_frame[y, 0:col_num-1] # get the row
                holder = np.sum(row == 255) # add to holder the sum of white pixels
                if holder > threshold:
                    message_list.append(1)
                else:
                    message_list.append(0)
            
            # plt.plot(new)
            # plt.ylabel('Number of white pixels')
            # plt.xlabel('Row number')
            # plt.show()
            # print(useful_frame.shape) # just testing
            
            # check for end sequence the last 30 bits
            # if it's the end end = 1
            if len(message_list) >= N:
                if end_sequence == message_list[-N:]:
                    end = 1
        



        # just testing
        # cv2.imshow('frame', bnwFrame)
         
    # Press Q on keyboard to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
 
# Break the loop
    else:
        break

# When everything done, release
# the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

# extract the last 30 bits(the end sequence)
n = len(end_sequence)

# ✅ Remove the last N elements from a list (list slicing)
final_message = message_list[:len(message_list) - n]
# the message that was sent so that we can compare it to the received version
message = np.zeros(1500)
for i in range(message.size):
  if (i % 2) == 0:
    message[i] = 1
errors = []
if message == final_message:
    print("Message received successfully")
else:
    for i in range(len(final_message)):
        if final_message[i] != message[i]:
            errors.append(i)
print("The errors are at the positions:")
print(errors)

print("Total errors are:")
print(len(errors))

print("Error percentage:")
print(len(errors)/len(message))

# convert binary to jpg, png, mp3, raw, text...
# show the received message

KeyboardInterrupt: 

# **Compare lists test**

In [8]:
start_sequence = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
temp = []
for i in range(29):
    temp.append(1)
temp.append(0)
N = 5

if start_sequence == temp:
    print("equal")
elif start_sequence != temp:
    print("not equal")
a = start_sequence[-N:]
print(a)
print(len(start_sequence))

equal
[1, 1, 1, 1, 0]
30


# **Sub-sampling test**

In [12]:
import cv2

# Open the video file
cap = cv2.VideoCapture('output.mp4')

y_min = 60
y_max = 360
x_min = 360
x_max = 665
frame_width = x_max - x_min
frame_height = y_max - y_min

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('draw_lines.mp4', fourcc, 20.0, (frame_width, frame_height))

while cap.isOpened():
    print("ok")
    # Read a frame from the video
    ret, frame = cap.read()

    if ret:
        print("ok")
        # for each row with step == thickness
        thickness = 10 # pixels (change manually)
        threshold = 100 # pixels (change manually)
        for y in range(int(thickness/2), int(frame_height - thickness/2), thickness):
            cv2.line(frame, (y, 0), (y, frame.shape[0]), (0, 0, 255), 5)

        # Write the modified frame to the output video file
        out.write(frame)

        # Display the frame
        cv2.imshow('frame', frame)
        
        # Exit if 'q' is pressed
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

# Release the resources
cap.release()
out.release()
cv2.destroyAllWindows()

